In [1]:
#do this later from admin: !python -m pip install --upgrade pip



In [2]:
#Basic
import pandas as pd
import numpy as np
#Math
import scipy as sp
from random import randint
from math import exp
import operator

#Scraping
import requests
import json

#API Packages: 
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

#Modeling
#from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
#from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.cluster import KMeans
#from sklearn.svm import SVC
#from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

#NLP Processing
'''from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer'''

#Time analysis
import time
import datetime
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud

%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [3]:
my_api_file = open("./mykey.txt", 'r')
my_api_str = my_api_file.read()
my_api_file.close()

In [4]:
DEVELOPER_KEY = my_api_str
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [36]:
def youtube_search(q, max_results=50,order="relevance", token=None, location=None, location_radius=None):
    #see Attribution 1: YouTube data pull structure:  

    #Google has many apis. This function creates a structured request
    youtube = build("youtube", "v3", developerKey=my_api_str)

    search_response = youtube.search().list(
    q=q, #Search term
    type="video", #Not comments
    pageToken=token, #token is point in youtube list of relevant
    order = 'viewCount', #sortby feature
    part="id,snippet",
    maxResults=max_results,
    location=location, #optional
    locationRadius=location_radius #optional

  ).execute()

    videos = []
    for search_result in search_response.get("items", []):
        if search_result["id"]["kind"] == "youtube#video":
            videos.append(search_result)
    try:
        nexttok = search_response["nextPageToken"]
        return(nexttok, videos)
    except Exception as e:
        nexttok = "last_page"
        return(nexttok, videos)

In [37]:
test = youtube_search("spinners")

In [38]:
just_json = test[1]

In [39]:
print('    ', just_json[0])

     {'kind': 'youtube#searchResult', 'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/EiVVEoQg6-AM4fjWTMN1iYaFfVQ"', 'id': {'kind': 'youtube#video', 'videoId': 'EVBc3QxCE3I'}, 'snippet': {'publishedAt': '2017-12-04T09:31:06.000Z', 'channelId': 'UCUI8XDywSfFSYBLqNS79QKQ', 'title': 'Learn Colors with 3D Fidget Spinners for Kids', 'description': 'Toddler Learning Video Educational Fun and Creatives Videos Sounds : freesound.org/jokersounds.com/soundbible.com Licensed under Creative Commons: ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/EVBc3QxCE3I/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/EVBc3QxCE3I/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/EVBc3QxCE3I/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'Evan Kids', 'liveBroadcastContent': 'none'}}


If we send the token with the next request, YouTube will know where it left off, and send us the next 50 most-relevant results. Let’s try.

In [46]:
token = test[0]
youtube_search("spinners", token=token)

('CGQQAA',
 [{'kind': 'youtube#searchResult',
   'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/pWCaP4e_wrqQEzh5Q12JEfS55yk"',
   'id': {'kind': 'youtube#video', 'videoId': 'XIjRcsXqGF4'},
   'snippet': {'publishedAt': '2017-08-02T03:54:33.000Z',
    'channelId': 'UCrCRc33bHXw774mdv0alEkw',
    'title': 'Le REGALAMOS un FIDGET SPINNER a NIÑOS DE LA CALLE',
    'description': 'Deja tu hermosisimo like y tu suscripcion con campanita activada REGALANDO SPINNERS POR LAS CALLES REGALANDO SPINNERS *SIGUEME EN MI ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/XIjRcsXqGF4/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/XIjRcsXqGF4/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/XIjRcsXqGF4/hqdefault.jpg',
      'width': 480,
      'height': 360}},
    'channelTitle': 'DimeEliseo',
    'liveBroadcastContent': 'none'}},
  {'kind': 'youtube#searchResult',
   'etag': '"XI7nbFXul

Automate the process

In [ ]:
video_dict = {'youID':[], 'title':[], 'pub_date':[]}

def grab_videos(keyword, token=None):
    res = youtube_search(keyword, token=token)
    token = res[0]
    just_json = res[1]
    for vid in just_json:
        video_dict['youID'].append(vid['id']['videoId'])
        video_dict['title'].append(vid['snippet']['title'])
        video_dict['pub_date'].append(vid['snippet']['publishedAt'])
    print "added " + str(len(just_json)) + " videos to a total of " + str(len(video_dict['youID']))
    return token

In [ ]:
# all views over 400M https://www.youtube.com/playlist?list=PLirAqAtl_h2r5g8xGajEwdXd3x1sZh8hC

https://www.youtube.com/results?filters=video&lclk=video&search_query=a%7Cb%7Cc%7Cd%7Ce%7Cf%7Cg%7Ch%7Ci%7Cj%7Ck%7Cl%7Cm%7Cn%7Co%7Cp%7Cq%7Cr%7Cs%7Ct%7Cu%7Cv%7Cw%7Cx%7Cy%7Cz%7C1%7C1%7C2%7C3%7C4%7C5%7C6%7C7%7C8%7C9%7C0&search_sort=video_view_count


EYYYYYYYY
https://github.com/youtube/api-samples/blob/master/python/search.py


In [5]:
# This sample executes a search request for the specified search term.

import argparse

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [ ]:

#Attribution 
#https://stackoverflow.com/questions/48253741/youtube-api-v3-and-python-to-generate-list-of-views-likes-on-own-youtube-videos/52484656#52484656
#Sefo Noaman

In [ ]:
def youtube_search_list(q, max_results=50, sort_method='viewCount', token=None):
  # Call the search.list method to retrieve results matching the specified query term.
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

  # Call the search.list method to retrieve results matching the specified query term.
    search_response = youtube.search().list(
        q=q,
        part='id,snippet',
        pageToken=token,
        maxResults=max_results,
        order=sort_method
      ).execute()
    
    return search_response

In [108]:
def youtube_search_video(q='spinners', max_results=50, sort_method='viewCount', token=None):
    
    order = "viewCount"
    q=q
    max_results = max_results
    sort_method = sort_method
    token = token
    location = None
    location_radius = None
    
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,         
    developerKey=DEVELOPER_KEY)
    
    #Return list of matching records up to max_search
    search_result = youtube_search_list(q, max_results, sort_method, token)
    next_tok = search_result['nextPageToken']
    
    videos_list = []
    for search_result in search_result.get("items", []):
        
        if search_result["id"]["kind"] == 'youtube#video':
            temp_dict_ = {}
            #Available from initial search
            temp_dict_['title'] = search_result['snippet']['title']  
            temp_dict_['vidId'] = search_result['id']['videoId']  
            
            #Secondary call to find statistics results for individual video
            response = youtube.videos().list(
                part='statistics, snippet, contentDetails', 
                id=search_result['id']['videoId']
                    ).execute()  
            response_statistics = response['items'][0]['statistics']
            response_snippet = response['items'][0]['snippet']
            response_content= response['items'][0]['contentDetails']
            
            
            snippet_list = ['publishedAt','channelId', 'description', 
                            'channelTitle', 'tags', 'categoryId', 
                            'liveBroadcastContent', 'defaultLanguage', ]
            for val in snippet_list:
                try:
                    temp_dict_[val] = response_snippet[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'    
            
            stats_list = ['favoriteCount', 'viewCount', 'likeCount', 
                          'dislikeCount', 'commentCount']
            for val in stats_list:
                try:
                    temp_dict_[val] = response_statistics[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'
            
            for val in response_content.keys():
                try:
                    temp_dict_[val] = response_content[val]
                except:
                    #Not stored if not present
                    temp_dict_[val] = 'xxNoneFoundxx'
                    
            #add back to main list
            videos_list.append(temp_dict_)
            
    return videos_list, next_tok

In [ ]:
#Instatiate datframe using first results. 
#Allows for simple forloop
videos_list, next_tok = youtube_search_video('Music Video')
df_videos = pd.DataFrame(videos_list)
df_videos['request_token'] = next_tok

In [138]:
count=1
print('   next_tok', next_tok )

   next_tok CMgBEAA


In [145]:
pd.read_csv('./data/running_video_pull.csv').drop('Unnamed: 0', axis=1)

(147, 23)

In [146]:
df_videos = pd.read_csv('./data/running_video_pull.csv').drop('Unnamed: 0', axis=1)
next_tok = 'CMgBEAA'

In [147]:
while next_tok != 'last_page':

    videos_list, next_tok = youtube_search_video(q='Music Video', token=next_tok)
    print('    found page ', count)
    df_small_vids = pd.DataFrame(videos_list)
    df_small_vids['request_token'] = next_tok
    df_videos = pd.concat([df_videos, df_small_vids])
    count += 1
    if count % 3 == 0:
        df_videos.to_csv('./data/running_video_pull.csv', index=False)

    found page  2
    found page  3
    found page  4
    found page  5


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  import sys


    found page  6
    found page  7
    found page  8
    found page  9
    found page  10
    found page  11
    found page  12
    found page  13
    found page  14
    found page  15
    found page  16


KeyError: 'nextPageToken'

In [166]:
youtube_search_list(q='Music Video', max_results=20)

{'kind': 'youtube#searchListResponse',
 'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/43WDb_kvOeiUTLJQSV3wuJkUq8g"',
 'nextPageToken': 'CBQQAA',
 'regionCode': 'US',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 20},
 'items': [{'kind': 'youtube#searchResult',
   'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/RLOGPOWMrcSASLkOXWNqJqA_c6I"',
   'id': {'kind': 'youtube#video', 'videoId': 'kJQP7kiw5Fk'},
   'snippet': {'publishedAt': '2017-01-13T05:00:02.000Z',
    'channelId': 'UCLp8RBhQHu9wSsq62j_Md6A',
    'title': 'Luis Fonsi - Despacito ft. Daddy Yankee',
    'description': 'Despacito” disponible ya en todas las plataformas digitales: https://UMLE.lnk.to/DOoUzFp Sigue a Luis Fonsi: Official Site: http://www.luisfonsi.com/ Facebook: ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/kJQP7kiw5Fk/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/kJQP7kiw5Fk/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high'

In [163]:
df_videos['viewCount'] = df_videos['viewCount'].astype('int64')

In [165]:
df_videos['viewCount'].min()

399849

Ways to expand on data if youtube wants to stop here (says there are 1M results):
* Most viewed vevo videos of all time
* Country hits
* Sourt by relevance instead
* Branch out from artists in top scoring


In [168]:

df_videos.shape[0]

282

In [170]:
df_videos.shape[0] / .25

1128.0

In [117]:
'''Hold off on this one until I figure out how to tie in google trends. 
Probably we want only comments from peak viewership plus marks at half peak on either side. 
Right now, most relevant /recent comments are all stupid meme stuff, e.g. who is watching in 2018'''
def youtube_comment_list(video_id, max_results=50, sort_method='viewCount', token=None):
  # Call the search.list method to retrieve results matching the specified query term.
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

  # Call the search.list method to retrieve results matching the specified query term.
    search_response = youtube.commentThreads().list(
        videoId = video_id,
        part='snippet, replies',
        pageToken=token,
        order = 'relevance',
        maxResults=max_results,
      ).execute()
    
    return search_response

psy = youtube_comment_list('9bZkp7q19f0')

psy['items'][1]['snippet']['topLevelComment']['snippet']['textDisplay']